In [1]:
import socketserver
from Crypto.Cipher import AES
from Crypto.Util import Counter
import os
import numpy as np
import matplotlib.pyplot as plt

In [2]:
key = os.urandom(0x10).replace(b'\x00', b'\xff')
iv = os.urandom(0x10).replace(b'\x00', b'\xff')

flag = "HTB{1234567890}"

In [3]:
def xor(a, b):
    return bytes([_a ^ _b for _a, _b in zip(a, b)])

def unhex(msg):
    return bytes.fromhex(msg)

def clean_encrypt(data):
    ctr = Counter.new(128, initial_value=int(iv.hex(), 16))
    crypto = AES.new(key, AES.MODE_CTR, counter=ctr)
    if type(data) != bytes:
        data = data.encode()
    return crypto.encrypt(data)

def encrypt(data):
    ctr = Counter.new(128, initial_value=int(iv.hex(), 16))
    crypto = AES.new(key, AES.MODE_CTR, counter=ctr)
    if type(data) != bytes:
        data = data.encode()
    otp = os.urandom(len(data)).replace(b'\x00', b'\xff')
    return xor(crypto.encrypt(data), otp)

In [4]:
def pt2ct(pt: str) -> bytes:
    d = {pos: [] for pos in range(len(pt))}
    for _ in range(10_000):
        noisy_ct = encrypt(pt)
        for pos in range(len(pt)):
            d[pos].append(noisy_ct[pos])
    ct = b''
    for pos in range(len(pt)):
        u, counts = np.unique(d[pos], return_counts=True)
        ct += bytes([u[np.argmax(counts)] ^ 0xff])
    return ct

In [5]:
pt = "I_am_a_test_string"
pt2ct(pt) == clean_encrypt(pt)

True

In [6]:
zeros = b'\x00' * len(flag)
xor(pt2ct(zeros), pt2ct(flag))

b'HTB{1234567890}'

---

In [7]:
import socket
import re
import time
from tqdm.notebook import tqdm

addr = "167.99.85.216"
port = 31247
timeout = 0.01
N = 10_000

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect((addr, port))
time.sleep(0.1)
print(s.recv(0x1000).decode())

1) Get flag
2) Encrypt Message
3) Decrypt Message
Your option: 


In [8]:
def get_noisy_enc_flag(s):
    time.sleep(timeout)
    s.send(b'1\n')
    time.sleep(timeout)
    data = s.recv(0x1000).decode()
    noisy_enc_flag = re.search(r'[0-9a-f]+', data).group(0)
    time.sleep(timeout)
    s.recv(0x1000) # return to menu
    return bytes.fromhex(noisy_enc_flag)

def get_noisy_enc_zeros(s, LEN):
    time.sleep(timeout)
    s.send(b'2\n')
    time.sleep(timeout)
    s.recv(0x1000).decode()
    zeros = b'\x00' * LEN
    time.sleep(timeout)
    s.send(zeros.hex().encode() + b'\n')
    data = s.recv(0x1000).decode()
    noisy_enc_zeros = re.search(r'[0-9a-f]+', data).group(0)
    time.sleep(timeout)
    s.recv(0x1000) # return to menu
    return bytes.fromhex(noisy_enc_zeros)

In [9]:
FLAG_LEN = len(get_noisy_enc_flag(s))

In [10]:
d = {pos: [] for pos in range(FLAG_LEN)}
for _ in tqdm(range(N)):
    noisy_enc_flag = get_noisy_enc_flag(s)
    for pos in range(FLAG_LEN):
        d[pos].append(noisy_enc_flag[pos])
flag_ct = b''
for pos in range(FLAG_LEN):
    u, counts = np.unique(d[pos], return_counts=True)
    flag_ct += bytes([u[np.argmax(counts)] ^ 0xff])

  0%|          | 0/10000 [00:00<?, ?it/s]

In [11]:
flag_ct

b'\x9e\x04\xff\\\xca\xc60\x9f\x1a\x8e\x92\xf8\xa0|\x96'

In [12]:
d = {pos: [] for pos in range(FLAG_LEN)}
for _ in tqdm(range(N)):
    noisy_enc_zeros = get_noisy_enc_zeros(s, FLAG_LEN)
    for pos in range(FLAG_LEN):
        d[pos].append(noisy_enc_zeros[pos])
zeros_ct = b''
for pos in range(FLAG_LEN):
    u, counts = np.unique(d[pos], return_counts=True)
    zeros_ct += bytes([u[np.argmax(counts)] ^ 0xff])

  0%|          | 0/10000 [00:00<?, ?it/s]

In [13]:
zeros_ct

b"\xd6P\xbd'\xfb\x90o\xed)\xfb\xb6\xcb\x83]\xeb"

In [14]:
xor(flag_ct, zeros_ct)

b'HTB{1V_r3u$3#!}'